In [23]:
import argparse
from glob import glob
import os
import re
import difflib as dl
from tqdm import tqdm
import pandas as pd

In [28]:
parser = argparse.ArgumentParser()

parser.add_argument('--v1_path', default='s-kr/fine-tune/dataset/gec_dataset/gec_data0.tsv')

# v2는 정상-정상 데이터이므로 infer_path 없음
parser.add_argument('--v2_path', default='data/v1.0/north_phonetic_corpus_v2.tsv')

parser.add_argument('--v3_path', default='data/v1.0/north_phonetic_corpus_v1.tsv')

parser.add_argument('--v4_infer_path', default='/home/n-kr/dataset/tongilboo_infer_save')
parser.add_argument('--v4_label_path', default='/home/n-kr/dataset/tongilboo')

args = parser.parse_args([])

In [25]:
def align(inf_arr, lab_arr):
    if len(inf_arr) == 0 and len(lab_arr) > 0:
        inf_arr = lab_arr
        return inf_arr, lab_arr
    elif len(inf_arr) > 0 and len(lab_arr) == 0:
        lab_arr = inf_arr
        return inf_arr, lab_arr
                                  
    # 추론 텍스트와 레이블 텍스트의 길이 차이에 따른 비교
    if len(inf_arr) > len(lab_arr):
        tmp_arr = []
        tmp_arr.append(inf_arr[0] + inf_arr[1]) # 왜 ?
        if len(inf_arr) >= 3:
            tmp_arr.extend(inf_arr[2:])
            inf_arr = tmp_arr
        else:
            inf_arr = tmp_arr
    elif len(inf_arr) < len(lab_arr):
        tmp_arr = []
        tmp_arr.append(lab_arr[0] + lab_arr[1])
        if len(lab_arr) >= 3:
            tmp_arr.extend(lab_arr[2:])
            lab_arr = tmp_arr
        else:
            lab_arr = tmp_arr
    elif len(inf_arr) == len(lab_arr):
        return inf_arr, lab_arr
    
    return align(inf_arr, lab_arr)

In [26]:
# def overlap(text_arr):
#     overlap_text_arr = []
    
#     if len(text_arr) // 7 > 0:
#         n = len(text_arr) // 7
#     else:
#         n = 1
        
#     for w in range(n):
#         if w == 0:
#             tmp_res = text_arr[:10]
#             overlap_text_arr.append(" ".join(tmp_res))
#         else:
#             tmp_res = text_arr[w * 7: w * 7 + 10]
#             overlap_text_arr.append(" ".join(tmp_res))

#     return overlap_text_arr

def overlap(text_arr):
    overlap_text_arr = []
    
    if len(text_arr) // 7 > 0: # 왜 7 ????
        n = len(text_arr) // 7
    else:
        n = 1
        
    for j in range(n):
        if j == 0:
            tmp_res = text_arr[:10]
            overlap_text_arr.append(" ".join(tmp_res))
        else:
            if j != n - 1:
                tmp_res = text_arr[j * 7: j * 7 + 10]
                overlap_text_arr.append(" ".join(tmp_res))
            else:
                tmp_res = text_arr[j * 7:]
                overlap_text_arr.append(" ".join(tmp_res))  
        # print(tmp_res)

    return overlap_text_arr

In [35]:
import os
print(os.getcwd())

/gec_s/gec_s


### **1. 구화 데이터 wav2vec2 추론 phonetic 코퍼스 생성(비정상-정상)**

In [48]:
df = pd.read_csv('gec_data0.tsv', sep='\t',index_col='Unnamed: 0')

In [43]:
df_1.head()

,form,corrected_form
0,아 뭔 소리야 그건 또,"아 몬 소리야, 그건 또."
1,나는 학습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...,나는 악습은 원래 없어진다 없어져야 된다고 생각하긴 했는데 근데 그 약간 필요악으로...
2,그래서 지오랑 계산 올라가서 막 위에 운동하는 기구 있대요 그서 그걸로 운동할려고요,그래서 지호랑 계단 올라와서 막 위에 운동하는 기구 있대요. 그서 그걸로 운동 할려구요.
3,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야,그게 영 점 일 프로 가정의 아이들과 가정의 모습이야?
4,그 친해하는 판산님께라는 법 법 관련 드라마 알고 있어,그 친애하는 판사님께라는 법 법 관련 드라마 알고 있어?


In [44]:
df_1.tail()

,form,corrected_form
9995,앞다릿살 무시해,앞다릿살 무시해?
9996,나도 나도 학교 다닐 때는 일 년에 진짜 책을 한 권도 안 읽을 정도로 책을 별로 ...,나도 나도 학교 다닐 때는 일 년에 진짜 책을 한 권도 안 읽을 정도로 책을 별로 ...
9997,어 그래서 한국도 어디 나가서 돈 벌러 나가면은 그렇게 이상하게 쳐다 안 봐 나 일...,어. 그래서 한국도 어디 나가서 돈 벌러 나가면은 그렇게 이상하게 쳐다 안 봐. 나...
9998,나도 안 봤어,나도 안 봤어.
9999,내 복 찾아들렸는데 저도 그냥,내복 사다 드렸는데 저도 그냥.


In [46]:
# Importing libraries
import argparse
import re
import os
import random
from glob import glob

from datasets import load_metric
from datasets import Dataset, DatasetDict
import numpy as np
import pandas as pd
from setproctitle import setproctitle
from tqdm import tqdm

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, EarlyStoppingCallback


# for kobert
# from kobert_tokenizer import KoBERTTokenizer
# from transformers import BertModel

import torch

import nltk
# nltk.download('punkt')
import string
from transformers import AutoTokenizer

parser = argparse.ArgumentParser()
# parser.add_argument('--dataset_path', default='/gec/data/v1.0/clean_data_v2_utf8.tsv')
# parser.add_argument('--dataset_path', default='/home/gec_sj/gec/data/v1.0/Korean_corpus_2021.csv')
# parser.add_argument('--north_phonetic_data_v1_path', default='/home/gec_sj/gec/data/v1.0/north_phonetic_corpus_v1.tsv')
# parser.add_argument('--north_phonetic_data_v2_path', default='/home/gec_sj/gec/data/v1.0/north_phonetic_corpus_v2.tsv')
# parser.add_argument('--modu_spelling_data_path', default='/home/gec_sj/gec/data/v1.0/modu_spelling_corpus_2021.tsv')
# parser.add_argument('--kspon_spelling_data_path', default='/home/gec_sj/gec/data/v1.0/kspon_spelling_corpus.tsv')

parser.add_argument('--dataset_format', default='tsv')
# parser.add_argument('--pretrained_model', default='KETI-AIR/ke-t5-large')
parser.add_argument('--pretrained_model', default='KETI-AIR/ke-t5-large-ko')
# parser.add_argument('--pretrained_model', default='skt/kobert-base-v1')
# parser.add_argument('--pretrained_model', default='google/mt5-large')
parser.add_argument('--seed', default=68)
parser.add_argument('--process_name', default='mario-t5-large-ko_n1_n2')
parser.add_argument('--batch_size', default=4)
parser.add_argument('--devices', default='1,2')
parser.add_argument('--data_limit', default=False)
parser.add_argument('--test_size_ratio', default=0.01)
parser.add_argument('--output_dir', default="./results-64/")
parser.add_argument('--max_input_length', default=64)
parser.add_argument('--min_input_length', default=5)
parser.add_argument('--max_target_length', default=64)
parser.add_argument('--min_target_length', default=5)

args = parser.parse_args(args=[])

random.seed(args.seed)

os.environ["WANDB_DISABLED"] = "true"

setproctitle(args.process_name)
## cuda device 설정
os.environ["CUDA_VISIBLE_DEVICES"]= args.devices
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   #출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

# path = 's-kr/fine-tune/dataset/gec_dataset'

# file_list=glob(os.path.join(path, '*.tsv'))
# df32=pd.DataFrame(list())
# for path_file in tqdm(file_list):
#     s_df = pd.read_csv(path_file, sep='\t', encoding='utf-8',index_col='Unnamed: 0', error_bad_lines=False, engine='python')
#     df32=pd.concat([df32, s_df])
    



Device: cuda
Count of using GPUs: 2
Current cuda device: 0


0it [00:00, ?it/s]


In [9]:
path = 's-kr/fine-tune/dataset/gec_dataset'

file_list=glob(os.path.join(path, '*.tsv'))
file_list

[]

In [ ]:
df=df32.reset_index()
df=df[['form','corrected_form']]
df=df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df=df.sample(n=520000, replace=False) # 전체 60만개에서 52만개만 무작위로 가져옴
df['form'][:300000]=df.corrected_form[:300000] # 그중 30만개는 input과 output이 같은걸로 세팅
df=pd.DataFrame(df)


df['form'] = df['form'].apply(lambda x: str(x).strip())
df['corrected_form'] = df['corrected_form'].apply(lambda x: str(x).strip())

In [49]:
# regex = r'[^%!?~,.a-zA-Z0-9가-힣\s+]'
regex = r'[^가-힣.~!?,\s+]'

In [51]:
df['form'] = df['form'].apply(lambda x: re.sub(regex, '', x))
df['corrected_form'] = df['corrected_form'].apply(lambda x: re.sub(regex, '', x))

df['form'] = df['form'].apply(lambda x: x.strip())
df['corrected_form'] = df['corrected_form'].apply(lambda x: x.strip())

trgs = ['\xa0', '\\xa0', u'\xa0', u'\u200a', u'\u3000', '+']
def rm_trgs(x):
    for trg in trgs:
        if trg in x:
            x = x.replace(trg, '')
    return x

df['form'] = df['form'].apply(rm_trgs)
df['corrected_form'] = df['corrected_form'].apply(rm_trgs)


#불필요 공백, 특수문자 처리
def clean_space(x):
    x = re.sub(r'\n+\t+', ' ', x)
    x = re.sub(r'\s+', ' ', x) #remove extra space
    x = x.strip()
    # x = re.sub(r'^\s+', '', x) #remove space from start
    # x = re.sub(r'\s+$', '', x) #remove space from the end
    return x

df['form'] = df['form'].apply(clean_space)
df['corrected_form'] = df['corrected_form'].apply(clean_space)

fn_df = df[df.form.apply(lambda x: args.min_input_length < len(x) <= args.max_input_length)]
fn_df = df[df.corrected_form.apply(lambda x: args.min_target_length < len(x) <= args.max_target_length)]

In [52]:
infer_text_arr = fn_df['form'].to_list()
label_text_arr = fn_df['corrected_form'].to_list()

In [53]:
infer_arr = []
label_arr = []

for it, lt in tqdm(list(zip(infer_text_arr, label_text_arr))):
    try:
        it = it.replace('\n', ' ')
        it = re.sub('[\s]+', ' ', it)
        it = it.strip()
        # it = it.replace(' ', '\n')
        it = it.split()

        lt = lt.replace('\n', ' ')
        lt = re.sub('[\s]+', ' ', lt)
        lt = lt.strip()
        # lt = lt.replace(' ', '\n')
        lt = lt.split()
    except:
        continue

    for tag, i1, i2, j1, j2  in dl.SequenceMatcher(None, it, lt).get_opcodes():
        # print(f'{tag:7}   it[{i1}:{i2}] --> l2[{j1}:{j2}] {it[i1:i2]!r:>6} --> {lt[j1:j2]!r}')
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {it[i1:i2]!r} --> {lt[j1:j2]!r}')
        # infer_arr.append(it[i1:i2])
        # label_arr.append(lt[j1:j2])

        inf_arr = it[i1:i2]
        lab_arr = lt[j1:j2]

        inf_arr, lab_arr = align(inf_arr, lab_arr)
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {len(inf_arr)} --> {len(lab_arr)}')
        infer_arr.extend(inf_arr)
        label_arr.extend(lab_arr)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7906/7906 [00:00<00:00, 14312.57it/s]


In [54]:
len(infer_arr)

59190

In [55]:
len(label_arr)

59190

In [61]:
infer_arr[:5]

['아', '뭔', '소리야', '그건', '또']

In [62]:
label_arr[:5]

['아', '몬', '소리야,', '그건', '또.']

In [63]:
infer_ng_arr = overlap(infer_arr)
label_ng_arr = overlap(label_arr)

In [65]:
infer_arr

['아',
 '뭔',
 '소리야',
 '그건',
 '또',
 '그래서',
 '지오랑',
 '계산',
 '올라가서',
 '막',
 '위에',
 '운동하는',
 '기구',
 '있대요',
 '그서',
 '그걸로',
 '운동할려고요',
 '그게',
 '영',
 '점',
 '일',
 '프로',
 '가정의',
 '아이들과',
 '가정의',
 '모습이야',
 '그',
 '친해하는',
 '판산님께라는',
 '법',
 '법',
 '관련',
 '드라마',
 '알고',
 '있어',
 '나도',
 '몰라',
 '나',
 '그',
 '시리즈',
 '하나도',
 '안',
 '봤음',
 '어',
 '아',
 '내일',
 '나',
 '알바하구나',
 '아',
 '근데',
 '한',
 '시에',
 '닫는',
 '게',
 '좀',
 '아쉽긴',
 '한데',
 '거기',
 '진짜',
 '괜찮은데',
 '맞아',
 '시간',
 '안에',
 '풀고',
 '약간',
 '이것부터',
 '풀고',
 '요런식으로',
 '풀어보렴',
 '이런',
 '거',
 '삼',
 '점대야',
 '언니가',
 '한',
 '한',
 '한',
 '시간에',
 '이만',
 '만',
 '원',
 '거의',
 '이',
 '정도로고',
 '이',
 '정도란',
 '말야',
 '아',
 '근데',
 '강남',
 '너무',
 '비싸',
 '나',
 '오늘',
 '집에',
 '갈께',
 '해',
 '아',
 '뭔가',
 '효과가',
 '있어',
 '근데',
 '무한도전',
 '안전',
 '폐주돼서',
 '없어졌는데',
 '너는',
 '무한도전을',
 '자주',
 '잘',
 '챙겨',
 '봤어',
 '나',
 '다음',
 '주에',
 '강릉',
 '가는데',
 '같이',
 '가자',
 '야',
 '너',
 '데',
 '학교',
 '기숙사에서',
 '자면',
 '안',
 '되냐',
 '목보',
 '너무',
 '비싸',
 '어',
 '시외버스도',
 '비싸고',
 '한',
 '삼만',
 '원인가',
 '그럴걸',
 '근데',
 '왜',
 

In [66]:
infer_ng_arr

['아 뭔 소리야 그건 또 그래서 지오랑 계산 올라가서 막',
 '계산 올라가서 막 위에 운동하는 기구 있대요 그서 그걸로 운동할려고요',
 '그서 그걸로 운동할려고요 그게 영 점 일 프로 가정의 아이들과',
 '프로 가정의 아이들과 가정의 모습이야 그 친해하는 판산님께라는 법 법',
 '판산님께라는 법 법 관련 드라마 알고 있어 나도 몰라 나',
 '나도 몰라 나 그 시리즈 하나도 안 봤음 어 아',
 '봤음 어 아 내일 나 알바하구나 아 근데 한 시에',
 '근데 한 시에 닫는 게 좀 아쉽긴 한데 거기 진짜',
 '한데 거기 진짜 괜찮은데 맞아 시간 안에 풀고 약간 이것부터',
 '풀고 약간 이것부터 풀고 요런식으로 풀어보렴 이런 거 삼 점대야',
 '거 삼 점대야 언니가 한 한 한 시간에 이만 만',
 '시간에 이만 만 원 거의 이 정도로고 이 정도란 말야',
 '이 정도란 말야 아 근데 강남 너무 비싸 나 오늘',
 '비싸 나 오늘 집에 갈께 해 아 뭔가 효과가 있어',
 '뭔가 효과가 있어 근데 무한도전 안전 폐주돼서 없어졌는데 너는 무한도전을',
 '없어졌는데 너는 무한도전을 자주 잘 챙겨 봤어 나 다음 주에',
 '나 다음 주에 강릉 가는데 같이 가자 야 너 데',
 '야 너 데 학교 기숙사에서 자면 안 되냐 목보 너무',
 '되냐 목보 너무 비싸 어 시외버스도 비싸고 한 삼만 원인가',
 '한 삼만 원인가 그럴걸 근데 왜 그 집을 기억을 못',
 '집을 기억을 못 해 복싱 안 배워봤어 아 니가 사주는',
 '아 니가 사주는 거였어 아 그건 절대로 안 할 거야',
 '안 할 거야 그렇긴 했는데 아 진짜 그러 에씨 좋은',
 '그러 에씨 좋은 학교 다니니까 그런 것도 배우고 부럽다 불편하잖아',
 '배우고 부럽다 불편하잖아 장갑 응 근데 이거 한 세 시에',
 '한 세 시에 끝나니까 할머니가 아까 너가 말한 거 같은',
 '말한 거 같은 복숭아 뭐 그런 요플레 그걸로 해서 드셨던',
 '그걸로 해서 드셨던 거 같애 어 배운거 연습 연습할 시간이',


In [15]:
# infer_ng_arr = []
# label_ng_arr = []
# for i in tqdm(range(len(infer_arr) - 9)):
#     if i == 0:
#         tmp_arr = []
#         for word in infer_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))
    
#     if i % 7 == 0:
#         tmp_arr = []
#         for word in infer_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))

In [16]:
len(infer_ng_arr)

46435

In [17]:
len(label_ng_arr)

46435

In [18]:
infer_ng_arr[:5]

['높은 신준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화주니다 일본의 폭력적인',
 '화주니다 일본의 폭력적인 역사지우기 행태를 더 이상 보검한 있지 않을',
 '보검한 있지 않을 것이며 피해자들의 명예와 인권이 해복될 때까지 싸울',
 '해복될 때까지 싸울 것이라고 성명은 강조했습니다 이만 올해가 너무 흥치수럽고',
 '올해가 너무 흥치수럽고 너무 기쁜 김에 춤을 좋습니다 조선노동당 출판사에서는']

In [19]:
label_ng_arr[:5]

['높은 수준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화선식으로해나가는 일본의 폭력적인',
 '화선식으로해나가는 일본의 폭력적인 역사지우기 행태를 더 이상 보고만 있지 않을',
 '보고만 있지 않을 것이며 피해자들의 명예와 인권이 회복될 때까지 싸울',
 '회복될 때까지 싸울 것이라고 성명은 강조했습니다 하지만 올해가 너무 흥취스럽고',
 '올해가 너무 흥취스럽고 너무 기쁜 김에 춤을 췄습니다 조선노동당 출판사에서는']

In [20]:
data = {'form': infer_ng_arr, 'corrected_form': label_ng_arr}
df_1 = pd.DataFrame(data)

In [21]:
df_1.head()

,form,corrected_form
0,높은 신준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화주니다 일본의 폭력적인,높은 수준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화선식으로해나가는 일본...
1,화주니다 일본의 폭력적인 역사지우기 행태를 더 이상 보검한 있지 않을,화선식으로해나가는 일본의 폭력적인 역사지우기 행태를 더 이상 보고만 있지 않을
2,보검한 있지 않을 것이며 피해자들의 명예와 인권이 해복될 때까지 싸울,보고만 있지 않을 것이며 피해자들의 명예와 인권이 회복될 때까지 싸울
3,해복될 때까지 싸울 것이라고 성명은 강조했습니다 이만 올해가 너무 흥치수럽고,회복될 때까지 싸울 것이라고 성명은 강조했습니다 하지만 올해가 너무 흥취스럽고
4,올해가 너무 흥치수럽고 너무 기쁜 김에 춤을 좋습니다 조선노동당 출판사에서는,올해가 너무 흥취스럽고 너무 기쁜 김에 춤을 췄습니다 조선노동당 출판사에서는


In [22]:
df_1.tail()

,form,corrected_form
46430,휴지 자투리들을 쓸모없이 버렸는데 우리는 어떻게하면 이 전에는휴창생산공정에서나오는수...,휴지 자투리들을 쓸모없이 버렸는데 우리는 어떻게하면 이 부사물 가지고 이
46431,전에는휴창생산공정에서나오는수지자투이들을쓸없없이버려는데우리는어떻게 하면 이 부산을가지...,부사물 가지고 이 질좋은 건제품을 만들수없겠는가하는 착상을 모으던 해방 후
46432,목다 해방 후 우리 수령님께서 사회주의 강국건설의 최후승리에대한 시 보도를,모으던 해방 후 우리 수령님께서 사회주의 강국건설의 최후승리에대한 신심과 보도를
46433,최후승리에대한 시 보도를 마칩니다 세계의믿음성을 담발할 시 있게 되었고 진동,최후승리에대한 신심과 보도를 마칩니다 기능성을 담보할 수 있게 되었고 진동
46434,있게 되었고 진동 및 충격시험장치와온수도시험장치를 새롭게 제작 완성습다 꿩과 노르를 만리,있게 되었고 진동 및 충격실험장치를 새롭게 제작 완성해서는 꿩과 노루를많이길러산에놓...


### **2. 통일부 데이터 phonetic 코퍼스 생성(정상-정상)**

In [23]:
df_2 = pd.read_csv(args.v2_path, sep='\t', encoding='utf-8')

In [24]:
df_2.head()

,form,corrected_form
0,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할 것을 미국...,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할 것을 미국...
1,미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은 자기의 미사일 계획에...,미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은 자기의 미사일 계획에...
2,이란의 국방력 강화가 미국의 지위를 더욱 약화시키고 있다고 하면서 그는 미군이 중동...,이란의 국방력 강화가 미국의 지위를 더욱 약화시키고 있다고 하면서 그는 미군이 중동...
3,미국의 콜로라도주에서 최근 사십대의 남성이 여러 명의 어린이들에게 성폭행을 가한 사...,미국의 콜로라도주에서 최근 사십대의 남성이 여러 명의 어린이들에게 성폭행을 가한 사...
4,평안북도에서 영농전투로 들끓는 농촌들에 대한 지원사업을 군중적 운동으로 힘있게 벌이...,평안북도에서 영농전투로 들끓는 농촌들에 대한 지원사업을 군중적 운동으로 힘있게 벌이...


In [25]:
df_2.tail()

,form,corrected_form
47206,성명은 사삼 봉기의 아픈 상처를 통탄하기 위해서는 보안법과 같은 반인권법을 폐지해야...,성명은 사삼 봉기의 아픈 상처를 통탄하기 위해서는 보안법과 같은 반인권법을 폐지해야...
47207,중국 외교부 대변인 류깡이 육일 미국의 추가적인 도발행위에 대응하려는 정부의 입장을...,중국 외교부 대변인 류깡이 육일 미국의 추가적인 도발행위에 대응하려는 정부의 입장을...
47208,미국이 천억달라어치의 중국 상품에 대해 새로운 추가관세를 부과하려고 시도하고 있는 ...,미국이 천억달라어치의 중국 상품에 대해 새로운 추가관세를 부과하려고 시도하고 있는 ...
47209,그는 미국의 언행을 지켜볼 것이라고 하면서 만약 미국이 중국과 국제사회의 반대에도 ...,그는 미국의 언행을 지켜볼 것이라고 하면서 만약 미국이 중국과 국제사회의 반대에도 ...
47210,미국이 침략,미국이 침략


In [26]:
infer_text_arr = df_2['form'].to_list()
label_text_arr = df_2['corrected_form'].to_list()

In [27]:
infer_arr = []
label_arr = []

for it, lt in tqdm(list(zip(infer_text_arr, label_text_arr))):
    try:
        it = it.replace('\n', ' ')
        it = re.sub('[\s]+', ' ', it)
        it = it.strip()
        # it = it.replace(' ', '\n')
        it = it.split()

        lt = lt.replace('\n', ' ')
        lt = re.sub('[\s]+', ' ', lt)
        lt = lt.strip()
        # lt = lt.replace(' ', '\n')
        lt = lt.split()
    except:
        continue

    for tag, i1, i2, j1, j2  in dl.SequenceMatcher(None, it, lt).get_opcodes():
        # print(f'{tag:7}   it[{i1}:{i2}] --> l2[{j1}:{j2}] {it[i1:i2]!r:>6} --> {lt[j1:j2]!r}')
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {it[i1:i2]!r} --> {lt[j1:j2]!r}')
        # infer_arr.append(it[i1:i2])
        # label_arr.append(lt[j1:j2])

        inf_arr = it[i1:i2]
        lab_arr = lt[j1:j2]

        inf_arr, lab_arr = align(inf_arr, lab_arr)
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {len(inf_arr)} --> {len(lab_arr)}')
        infer_arr.extend(inf_arr)
        label_arr.extend(lab_arr)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47211/47211 [00:04<00:00, 10901.03it/s]


In [28]:
len(infer_arr)

995508

In [29]:
len(label_arr)

995508

In [30]:
infer_arr[:5]

['이란', '무역', '총지휘', '참모부', '부참모장']

In [31]:
label_arr[:5]

['이란', '무역', '총지휘', '참모부', '부참모장']

In [32]:
infer_ng_arr = overlap(infer_arr)
label_ng_arr = overlap(label_arr)

In [33]:
# infer_ng_arr = []
# label_ng_arr = []
# for i in tqdm(range(len(infer_arr) - 9)):
#     if i == 0:
#         tmp_arr = []
#         for word in infer_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))
    
#     if i % 7 == 0:
#         tmp_arr = []
#         for word in infer_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))

In [34]:
len(infer_ng_arr)

142215

In [35]:
len(label_ng_arr)

142215

In [36]:
data = {'form': infer_ng_arr, 'corrected_form': label_ng_arr}
df_2 = pd.DataFrame(data)

In [37]:
df_2.head()

,form,corrected_form
0,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할
1,장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에 요구했습니다 미국과 유럽이,장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에 요구했습니다 미국과 유럽이
2,요구했습니다 미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은,요구했습니다 미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은
3,않는 한 이란은 자기의 미사일 계획에 관한 협상에 나서지 않을,않는 한 이란은 자기의 미사일 계획에 관한 협상에 나서지 않을
4,협상에 나서지 않을 것이라고 천명하고 그는 미국이 이란의 미사일 능력을,협상에 나서지 않을 것이라고 천명하고 그는 미국이 이란의 미사일 능력을


In [38]:
df_2.tail()

,form,corrected_form
142210,명확히 천명되었다고 말했습니다 그는 미국의 언행을 지켜볼 것이라고 하면서 만약,명확히 천명되었다고 말했습니다 그는 미국의 언행을 지켜볼 것이라고 하면서 만약
142211,것이라고 하면서 만약 미국이 중국과 국제사회의 반대에도 불구하고 일방주의적으로 보호...,것이라고 하면서 만약 미국이 중국과 국제사회의 반대에도 불구하고 일방주의적으로 보호...
142212,불구하고 일방주의적으로 보호무역주의적인 행위를 고집한다면 중국은 그 어떤 대가를 치...,불구하고 일방주의적으로 보호무역주의적인 행위를 고집한다면 중국은 그 어떤 대가를 치...
142213,어떤 대가를 치러서라도 반드시 이에 단호히 대응할 것이며 나라와 인민의,어떤 대가를 치러서라도 반드시 이에 단호히 대응할 것이며 나라와 인민의
142214,것이며 나라와 인민의 이익을 견결히 수호할 것이라고 밝혔습니다 미국이 침략,것이며 나라와 인민의 이익을 견결히 수호할 것이라고 밝혔습니다 미국이 침략


### **3. 구화 데이터 소리자바 추론 phonetic 코퍼스 생성(비정상-정상)**

In [39]:
df_3 = pd.read_csv(args.v3_path, sep='\t')

In [40]:
df_3.head()

,form,corrected_form
0,높은 신준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화주니다,높은 수준의 공연이지만 공연도 혁명적으로 전투적으로 그야말로 화선식으로 해나가는
1,일본의 폭력적인 역사지우기 행태를 더 이상 보검한 있지 않을 것이며 피해자들의 명예...,일본의 폭력적인 역사지우기 행태를 더 이상 보고만 있지 않을 것이며 피해자들의 명예...
2,이만 올해가 너무 흥치수럽고 너무 기쁜 김에 춤을 좋습니다,하지만 올해가 너무 흥취스럽고 너무 기쁜 김에 춤을 췄습니다
3,조선노동당 출판사에서는 삼지연군을 훌륭히 꾸리기 위한 투쟁에로 전체 군대와 인민을 ...,조선노동당 출판사에서는 삼지연군을 훌륭히 꾸리기 위한 투쟁에로 전체 군대와 인민을 ...
4,피델 까스뜨로 루쓰 동지의 서거는 꾸바 혁명과 공동의 위업을 위하여 투쟁하는 두 나...,피델 까스뜨로 루쓰 동지의 서거는 꾸바 혁명과 공동의 위업을 위하여 투쟁하는 두 나...


In [41]:
df_3.tail()

,form,corrected_form
24424,해방 우리 수령님께서,해방 후 우리 수령님께서
24425,사회주의 강국건설의 최후승리에 대한 시,사회주의 강국건설의 최후승리에대한 신심과
24426,보도를 마칩니다,보도를 마칩니다
24427,세계의 믿음성을 담발할 시 있게 되었고 진동 및 충격시험장치와 온수도시험장치를 새롭...,기능성을 담보할 수 있게 되었고 진동 및 충격실험장치를 새롭게 제작 완성해서는
24428,꿩과 노르를 만리,꿩과 노루를 많이 길러 산에 놓아줄 데


In [42]:
infer_text_arr = df_2['form'].to_list()
label_text_arr = df_2['corrected_form'].to_list()

In [43]:
infer_arr = []
label_arr = []

for it, lt in tqdm(list(zip(infer_text_arr, label_text_arr))):
    try:
        it = it.replace('\n', ' ')
        it = re.sub('[\s]+', ' ', it)
        it = re.sub('[세계는.]', '', it)
        it = it.strip()
        # it = it.replace(' ', '\n')
        it = it.split()

        lt = lt.replace('\n', ' ')
        lt = re.sub('[\s]+', ' ', lt)
        lt = lt.strip()
        # lt = lt.replace(' ', '\n')
        lt = lt.split()
    except:
        continue

    for tag, i1, i2, j1, j2  in dl.SequenceMatcher(None, it, lt).get_opcodes():
        # print(f'{tag:7}   it[{i1}:{i2}] --> l2[{j1}:{j2}] {it[i1:i2]!r:>6} --> {lt[j1:j2]!r}')
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {it[i1:i2]!r} --> {lt[j1:j2]!r}')
        # infer_arr.append(it[i1:i2])
        # label_arr.append(lt[j1:j2])

        inf_arr = it[i1:i2]
        lab_arr = lt[j1:j2]

        inf_arr, lab_arr = align(inf_arr, lab_arr)
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {len(inf_arr)} --> {len(lab_arr)}')
        infer_arr.extend(inf_arr)
        label_arr.extend(lab_arr)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 142215/142215 [00:10<00:00, 14006.58it/s]


In [44]:
len(infer_arr)

1421982

In [45]:
len(label_arr)

1421982

In [46]:
infer_arr[:5]

['이란', '무역', '총지휘', '참모부', '부참모장']

In [47]:
label_arr[:5]

['이란', '무역', '총지휘', '참모부', '부참모장']

In [48]:
infer_ng_arr = overlap(infer_arr)
label_ng_arr = overlap(label_arr)

In [49]:
# infer_ng_arr = []
# label_ng_arr = []
# for i in tqdm(range(len(infer_arr) - 9)):
#     if i == 0:
#         tmp_arr = []
#         for word in infer_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))
    
#     if i % 7 == 0:
#         tmp_arr = []
#         for word in infer_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))

In [50]:
len(infer_ng_arr)

203140

In [51]:
len(label_ng_arr)

203140

In [52]:
infer_ng_arr[:5]

['이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할',
 '장거리 미사일들을 철폐할 장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에',
 '미국과 유럽 나라들에 요구했습니다 미국과 유럽이 요구했습니다 미국과 유럽이 핵무기와',
 '미국과 유럽이 핵무기와 장거리 미사일을 없애지 않 한 이란은 않',
 '한 이란은 않 한 이란은 자기의 미사일 획에 관한 협상에']

In [53]:
label_ng_arr[:5]

['이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할',
 '장거리 미사일들을 철폐할 장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에',
 '미국과 유럽 나라들에 요구했습니다 미국과 유럽이 요구했습니다 미국과 유럽이 핵무기와',
 '미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은 않는',
 '한 이란은 않는 한 이란은 자기의 미사일 계획에 관한 협상에']

In [54]:
data = {'form': infer_ng_arr, 'corrected_form': label_ng_arr}
df_3 = pd.DataFrame(data)

In [55]:
df_3.head()

,form,corrected_form
0,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할,이란 무역 총지휘 참모부 부참모장 삼일 핵무기와 장거리 미사일들을 철폐할
1,장거리 미사일들을 철폐할 장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에,장거리 미사일들을 철폐할 장거리 미사일들을 철폐할 것을 미국과 유럽 나라들에
2,미국과 유럽 나라들에 요구했습니다 미국과 유럽이 요구했습니다 미국과 유럽이 핵무기와,미국과 유럽 나라들에 요구했습니다 미국과 유럽이 요구했습니다 미국과 유럽이 핵무기와
3,미국과 유럽이 핵무기와 장거리 미사일을 없애지 않 한 이란은 않,미국과 유럽이 핵무기와 장거리 미사일을 없애지 않는 한 이란은 않는
4,한 이란은 않 한 이란은 자기의 미사일 획에 관한 협상에,한 이란은 않는 한 이란은 자기의 미사일 계획에 관한 협상에


In [56]:
df_3.tail()

,form,corrected_form
203135,미국이 중국과 국제사회의 반대에도 불구하고 일방주의적으로 보호무역주의적인 불구하고 ...,미국이 중국과 국제사회의 반대에도 불구하고 일방주의적으로 보호무역주의적인 불구하고 ...
203136,불구하고 일방주의적으로 보호무역주의적인 행위를 고집한다면 중국은 그 어떤 대가를 치...,불구하고 일방주의적으로 보호무역주의적인 행위를 고집한다면 중국은 그 어떤 대가를 치...
203137,어떤 대가를 치러서라도 어떤 대가를 치러서라도 반드시 이에 단호히 대응할,어떤 대가를 치러서라도 어떤 대가를 치러서라도 반드시 이에 단호히 대응할
203138,이에 단호히 대응할 것이며 나라와 인민의 것이며 나라와 인민의 이익을,이에 단호히 대응할 것이며 나라와 인민의 것이며 나라와 인민의 이익을
203139,나라와 인민의 이익을 견결히 수호할 것이라고 밝혔습니다 미국이 침략,나라와 인민의 이익을 견결히 수호할 것이라고 밝혔습니다 미국이 침략


### **4. 통일부 데이터 소라자바 추론 phonetic 코퍼스 생성(비정상-정상)**

In [57]:
infer_txt_arr = glob(os.path.join(args.v4_infer_path, '*.txt'))
label_txt_arr = glob(os.path.join(args.v4_label_path, '**', '*.txt'), recursive=True)

In [58]:
infer_text_arr = []
label_text_arr = []
for infer_txt in tqdm(infer_txt_arr):
    _, file_name = os.path.split(infer_txt)
    infer_txt_name = file_name.replace('.wav', '')
    for label_txt in label_txt_arr:
        _, label_txt_name = os.path.split(label_txt)
        if infer_txt_name == label_txt_name:
            try:
                with open(infer_txt, 'r', encoding='cp949') as f:
                    data = f.read()
                infer_text_arr.append(data)
                with open(label_txt, 'r', encoding='utf-8') as f:
                    data = f.read()
                label_text_arr.append(data)
            except:
                break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 813/813 [00:02<00:00, 398.73it/s]


In [59]:
len(infer_text_arr)

744

In [60]:
len(label_text_arr)

744

In [61]:
infer_arr = []
label_arr = []

for it, lt in tqdm(list(zip(infer_text_arr, label_text_arr))):
    it = it.replace('\n', ' ')
    it = re.sub('[\s]+', ' ', it)
    it = re.sub('[세계는.]', '', it)
    it = it.strip()
    # it = it.replace(' ', '\n')
    it = it.split()

    lt = lt.replace('\n', ' ')
    lt = re.sub('[\s]+', ' ', lt)
    lt = lt.strip()
    # lt = lt.replace(' ', '\n')
    lt = lt.split()

    for tag, i1, i2, j1, j2  in dl.SequenceMatcher(None, it, lt).get_opcodes():
        # print(f'{tag:7}   it[{i1}:{i2}] --> l2[{j1}:{j2}] {it[i1:i2]!r:>6} --> {lt[j1:j2]!r}')
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {it[i1:i2]!r} --> {lt[j1:j2]!r}')
        # infer_arr.append(it[i1:i2])
        # label_arr.append(lt[j1:j2])

        inf_arr = it[i1:i2]
        lab_arr = lt[j1:j2]

        inf_arr, lab_arr = align(inf_arr, lab_arr)
        # print(f'{tag:7}   it[{i1}:{i2}] --> lt[{j1}:{j2}] {len(inf_arr)} --> {len(lab_arr)}')
        infer_arr.extend(inf_arr)
        label_arr.extend(lab_arr)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 744/744 [00:15<00:00, 47.03it/s]


In [62]:
len(infer_arr)

891453

In [63]:
len(label_arr)

891453

In [64]:
infer_arr[:5]

['위대한', '영도자', '김정일', '동지에', '고전적로']

In [65]:
label_arr[:5]

['위대한', '영도자', '김정일', '동지의', '고전적']

In [66]:
infer_ng_arr = overlap(infer_arr)
label_ng_arr = overlap(label_arr)

In [67]:
# infer_ng_arr = []
# label_ng_arr = []
# for i in tqdm(range(len(infer_arr) - 9)):
#     if i == 0:
#         tmp_arr = []
#         for word in infer_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i:i+10]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))
    
#     if i % 7 == 0:
#         tmp_arr = []
#         for word in infer_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         infer_ng_arr.append(' '.join(tmp_arr))

#         tmp_arr = []
#         for word in label_arr[i-7:i+3]:
#             if word.strip() == '':
#                 continue
#             tmp_arr.append(word.strip())
#         label_ng_arr.append(' '.join(tmp_arr))

In [68]:
infer_ng_arr[5:10]

['김일성 동지께서 지금으로부터 50년 전인 주체 57 1968년 3월 14일',
 '1968년 3월 14일 불후의 고전적로 작 학생들을 사회주의 공산주의 건설에',
 '사회주의 공산주의 건설에 참된 후비대로 교육 교양하자를 발표하시었습니다 외침이 날에',
 '발표하시었습니다 외침이 날에 즈음해서 교육위원회 국장 박일 부국장 박태현 동지들은',
 '부국장 박태현 동지들은 이렇게 말했습니다 의 일찍이 혁명투쟁과건설사업에서학교 교육 사업을']

In [69]:
label_ng_arr[5:10]

['김일성 동지께서는 지금으로부터 오십년 전인 주체 오십칠천구백육십팔 년 삼월 십사일',
 '년 삼월 십사일 불후의 고전적 로작 학생들을 사회주의 공산주의 건설의',
 '사회주의 공산주의 건설의 참된 후비대로 교육교양하자 를 발표하시었습니다 이 날에',
 '발표하시었습니다 이 날에 즈음해서 교육위원회 국장 박일 부국장 박태윤 동지들은',
 '부국장 박태윤 동지들은 이렇게 말했습니다 의 일찍이 혁명투쟁과 건설사업에서 학교교육사업을']

In [70]:
len(infer_ng_arr)

127350

In [71]:
len(label_ng_arr)

127350

In [72]:
data = {'form': infer_ng_arr, 'corrected_form': label_ng_arr}
df_4 = pd.DataFrame(data)

In [73]:
df_4.head()

,form,corrected_form
0,위대한 영도자 김정일 동지에 고전적로 작을 네팔 신문이 게재했습니다 위대한,위대한 영도자 김정일 동지의 고전적 로작을 네팔 신문이 게재했습니다 위대한
1,신문이 게재했습니다 위대한 영도자 김정일 동지의 불후의 고전적로작만경대 혁명 학원은,신문이 게재했습니다 위대한 영도자 김정일 동지의 불후의 고전적 로작 만경대혁명학원은
2,고전적로작만경대 혁명 학원은 주체 혈통을 이어 나갈 핵심골간양성기지다의 전문을 네팔,고전적 로작 만경대혁명학원은 주체의 혈통을 이어 나갈 핵심골간양성기지이다 를 네팔
3,핵심골간양성기지다의 전문을 네팔 신문 아르판2 일부가 게재했습니다 신문은 위대한 장군님의,핵심골간양성기지이다 를 네팔 신문 아르판 이일부가 게재했습니다 신문은 위대한 장군님의
4,신문은 위대한 장군님의 영상사진문헌을 모시었습니다 위대한 수령 김일성 동지께서 지금...,신문은 위대한 장군님의 영상사진문헌을 모시었습니다 위대한 수령 김일성 동지께서는 지...


In [74]:
df_4.tail()

,form,corrected_form
127345,9도 영하 3도 강 영하 21도 영하 11도 혜산 영하,구도 영하 삼도 강계 영하 이영도 영하 일영도 혜산 영하
127346,11도 혜산 영하 22도 영하 13도 청진 영하 12도 영하,일영도 혜산 영하 이십이도 영하 십삼도 청진 영하 일영도 영하
127347,영하 12도 영하 4도 함흥 영하 10도 0도 원산 영하,영하 일영도 영하 사도 함흥 영하 일영도 영도 원산 영하
127348,0도 원산 영하 6도 영하 1도라선 영하 14도 영하 7도,영도 원산 영하 육도영하 일도 라선 영하 십사도 영하 칠도
127349,14도 영하 7도 남포 영하 10도 영하 6도 정도로 예견됩니다,십사도 영하 칠도 남포 영하 일영도 영하 육도 정도로 예견됩니다


In [75]:
df = pd.concat([df_1, df_2, df_3, df_4])
df = df.sample(frac=1).reset_index(drop=True)

In [76]:
len(df)

519140

In [77]:
df.head()

,form,corrected_form
0,토론했습니다 토론자들은 위대한 영도자 김정일 동지 영도자 김정일 동지 인민대중의,토론했습니다 토론자들은 위대한 영도자 김정일 동지는 영도자 김정일 동지는 인민대중의
1,깊이 되새겨보고 있습니다 영웅적 되새겨보고 있습니다 영웅적 조선인민군은 절 위인들에,깊이 되새겨보고 있습니다 영웅적 되새겨보고 있습니다 영웅적 조선인민군은 절세 위인들에
2,친선협조관를 발전시키려 의지가 담겨져 있다고 한 전국위원장의 말을 전달했습니다 전국...,친선협조관계를 발전시키려는 의지가 담겨져 있다고 한 전국위원장의 말을 전달했습니다 ...
3,법외노조화의 철회가 민주주의 회복의 상징인데도 당국은 삼십여차례의 협의과정에서 말바...,법외노조화의 철회가 민주주의 회복의 상징인데도 당국은 삼십여차례의 협의과정에서 말바...
4,그때부터 지난해 십일월까지 백두산 절세 위인들을 칭송하고 선군정치를 소개하는 글들,그때부터 지난해 십일월까지 백두산 절세 위인들을 칭송하고 선군정치를 소개하는 글들


In [78]:
df.tail()

,form,corrected_form
519135,자주 조직해서 모든 청소년학생들을 당의 모든 청소년학생들을 당의 영도에 끝없이,자주 조직해서 모든 청소년학생들을 당의 모든 청소년학생들을 당의 영도에 끝없이
519136,우리 당의 혁명사상인 주체사상과 당의 혁명사상인 주체사상과 백두의 혁명정신을 뼛속,우리 당의 혁명사상인 주체사상과 당의 혁명사상인 주체사상과 백두의 혁명정신을 뼛속
519137,있는 명작을 내놓기 위해서 심혈을 쏟아 붓고 있습니다 그런가 하면,있는 명작을 내놓기 위해서 심혈을 쏟아 붓고 있습니다 그런가 하면
519138,받들고 송림시의 여맹 일꾼들과 여맹원들이 주체철 생산투쟁으로 여맹원들이 주체철 생산...,받들고 송림시의 여맹 일꾼들과 여맹원들이 주체철 생산투쟁으로 여맹원들이 주체철 생산...
519139,이 아니십니까 바로 이 위대한 사명감을 안으시고 혁명영도의 전기간 온,이 아니십니까 바로 이 위대한 사명감을 안으시고 혁명영도의 전기간 온


In [79]:
df.to_csv('data/v2.0/n1_n2_n3_n4_corpus.tsv', sep='\t', encoding='utf-8')